In [ ]:
import pandas as pd
from dns.resolver import resolve, resolve_at
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import thread_map, process_map
tqdm.pandas()
# From https://host.io/rankings
df = pd.read_csv("host10m.csv.gz", names=["domain", "rank"]).head(100000)
df

In [ ]:
bad_IPs = "65.9.141.129,65.9.141.14,65.9.141.29,65.9.141.47,65.9.141.57,65.9.141.82,65.9.141.86,65.9.141.92".split(",")
bad_IPs

In [ ]:
def safe_resolve(domain):
  try:
    return resolve(domain)
  except Exception as e:
    print(e)
df["response"] = thread_map(safe_resolve, df.domain)
#df["response"] = df.domain.progress_apply(safe_resolve)

In [ ]:
def calc_failure_rate(response):
    if response:
        return sum([str(r) in bad_IPs for r in response])/len(response)

df["failure_rate"] = df.response.apply(calc_failure_rate)
df[df.failure_rate > 0].sort_values("failure_rate")

In [ ]:
df[df.failure_rate > .25]

In [ ]:
def ans_to_str(response):
  if response:
    return [str(a) for a in response]
df.response = df.response.apply(ans_to_str)
df.to_csv("100K_domains_resolved.csv", index=False)